In [ ]:
import pandas as pd
import requests
import zipfile
import io
import os
import csv
import ctypes as ct
import numpy as np
from collections import Counter
import matplotlib.pyplot as plt
import re


- "https://www.bankofengland.co.uk/-/media/boe/files/markets/asset-purchase-facility/gilt-purchase-operational-results.xlsx"
- "https://www.bankofengland.co.uk/-/media/boe/files/markets/asset-purchase-facility/gilt-sales-time-series.xlsx"

In [ ]:
df_buy = pd.ExcelFile("downloads/gilt-purchase-operational-results.XLSX")

In [ ]:
df_buy.sheet_names

In [ ]:
df_buy = pd.read_excel("downloads/gilt-purchase-operational-results.XLSX", sheet_name="APF Gilts", header=1)

In [ ]:
df_buy.columns

In [ ]:
df_boe = df_buy[['Operation date','Total allocation (proceeds £mn)', 'Total allocation (nominal £mn)']].copy()

In [ ]:
df_boe['Operation date'] = pd.to_datetime(df_boe['Operation date'], format='%d-%m-%Y')

In [ ]:
df_boe.set_index('Operation date', inplace=True)

In [ ]:
df_boe.cumsum(axis=0).plot()

In [ ]:
df_sell = pd.ExcelFile("downloads/gilt-sales-time-series.XLSX")

In [ ]:
df_sell.sheet_names

In [ ]:
df_sell = pd.read_excel("downloads/gilt-sales-time-series.XLSX", sheet_name="APF gilt sales", header=1)

In [ ]:
df_sell.columns

In [ ]:
df_boe_exit = df_sell[['Operation date','Total allocation (proceeds £mn)', 'Total allocation (nominal £mn)']].copy()
df_boe_exit['Operation date'] = pd.to_datetime(df_boe_exit['Operation date'], format='%d-%m-%Y')
df_boe_exit.set_index('Operation date', inplace=True)
df_boe_exit.cumsum(axis=0).plot()

In [ ]:
bonds = df_buy[['ISIN','Bond\n']].drop_duplicates()

In [ ]:
from collections import defaultdict
isin_bond_map = defaultdict(list)
for row in bonds.itertuples():
    isin_bond_map[row.ISIN].append(row._2)
for isin, bond in isin_bond_map.items():
    if len(bond) > 1:
        raise Exception(f"There are duplicates in the BoE list: {isin}")
isin_bond_map['GB0008881541']

In [ ]:
gilt_level_buys = df_buy[[
    'ISIN',
    'Operation date',
    'Total allocation (proceeds £mn)',
    'Total allocation (nominal £mn)'
    ]].copy()
gilt_level_buys.set_index('Operation date', inplace=True)
gilt_level_buys.index = pd.to_datetime(gilt_level_buys.index, format='%d-%m-%Y').to_period('M').to_timestamp('M')
gilt_level_buys.reset_index(inplace=True)


In [ ]:
gilt_buys_ts = gilt_level_buys.pivot_table(
    index='Operation date', 
    columns='ISIN', 
    values='Total allocation (nominal £mn)', aggfunc='sum')

In [ ]:
gilt_buys_ts.columns

In [ ]:
gilt_buys_ts = gilt_buys_ts.fillna(0).cumsum()
gilt_buys_ts.head()
# TODO: Clip once the Bond has matured

In [ ]:
gilt_buys_total = gilt_level_buys.reset_index().groupby([ 'ISIN']).agg(
    Total_Allocation_Proceeds=('Total allocation (proceeds £mn)', 'sum'),
    Total_Allocation_Nominal=('Total allocation (nominal £mn)', 'sum'),
    Last_Purchase_Month=('Operation date', 'max')
)
gilt_buys_total.reset_index(inplace=True)
gilt_buys_total.head()
    

In [ ]:
from bgs.load_gilt_details import load_csv_blocks
details = load_csv_blocks("downloads/BGSDetails.csv")

In [ ]:
conv_details.columns

In [ ]:
conv_details.head()

In [ ]:
conv_details.replace(to_replace=32112, value=32120, inplace=True)

In [ ]:
conv_details[conv_details['Sequence']== '32120']

In [ ]:
isin_bond_details = conv_details[['ISIN Code', 'Latest redemption date','%','Sequence']].copy()

In [ ]:
[x for x in gilt_buys_total['ISIN'] if x not in isin_bond_details['ISIN Code'].to_list()]

In [ ]:
gilt_buys_total['ISIN'][0]

In [ ]:
isin_bond_details[isin_bond_details['ISIN Code']=='GB0002404191']['Latest redemption date'].values[0]

In [ ]:
gilt_buys_total['maturity_date'] = gilt_buys_total['ISIN'].map(
    lambda x: isin_bond_details[
        isin_bond_details['ISIN Code']==x
        ]['Latest redemption date'].values[0]
    )
gilt_buys_total['bgs_id'] = gilt_buys_total['ISIN'].map(
    lambda x: isin_bond_details[
        isin_bond_details['ISIN Code']==x
        ]['Sequence'].values[0]
    )

In [ ]:
gilt_buys_total['maturity_date'] = pd.to_datetime(
    gilt_buys_total['maturity_date'], format="%d %b %Y"
)

In [ ]:
bond_sales = df_sell[['ISIN','Bond\n']].drop_duplicates()

In [ ]:
isin_bond_sales_map = defaultdict(list)
for row in bond_sales.itertuples():
    isin_bond_sales_map[row.ISIN].append(row._2)
for isin, bond in isin_bond_sales_map.items():
    if len(bond) > 1:
        raise Exception(f"There are duplicates in the BoE list: {isin}")
isin_bond_sales_map['GB0008881541']

In [ ]:
gilt_level_sales = df_sell[[
    'ISIN',
    'Operation date',
    'Total allocation (proceeds £mn)',
    'Total allocation (nominal £mn)'
    ]].copy()
gilt_level_sales.set_index('Operation date', inplace=True)
gilt_level_sales.index = pd.to_datetime(gilt_level_sales.index, format='%d-%m-%Y').to_period('M').to_timestamp('M')
gilt_level_sales.reset_index(inplace=True)
gilt_level_sales.head()

In [ ]:
sales_by_isin=gilt_level_sales.groupby('ISIN')[['Total allocation (nominal £mn)', 'Total allocation (proceeds £mn)']].sum().reset_index()

In [ ]:
sales_by_isin['maturity_date'] = sales_by_isin['ISIN'].map(
    lambda x: isin_bond_details[
        isin_bond_details['ISIN Code']==x
        ]['Latest redemption date'].values[0]
    )
sales_by_isin['maturity_date'] = pd.to_datetime(
    sales_by_isin['maturity_date'], format="%d %b %Y"
)

In [ ]:
matured_sales = sales_by_isin['maturity_date'] > pd.Timestamp('2025-04-30')

In [ ]:
matured = gilt_buys_total['maturity_date'] > pd.Timestamp.now()

In [ ]:
sales_by_isin[matured_sales].shape, gilt_buys_total[matured].shape

In [ ]:
check_sales = sales_by_isin[matured_sales]['Total allocation (nominal £mn)'].sum()
check_sales

In [ ]:
unsold = [x for x in gilt_buys_total[matured]['ISIN'].to_list() if x not in sales_by_isin[matured_sales]['ISIN'].to_list()]  
unsold

In [ ]:
conv_details[conv_details['ISIN Code']==unsold[0]].T

In [ ]:
boe_portfolio = gilt_buys_total[matured].set_index('ISIN').merge(
    sales_by_isin[matured_sales][[
        'ISIN',
        'Total allocation (nominal £mn)',
        'Total allocation (proceeds £mn)'
        ]].set_index('ISIN'),
        how='left',
        left_index=True,
        right_index=True,
        suffixes=('_buy', '_sell')
        ).copy().fillna(0).reset_index()

In [ ]:
boe_portfolio.head()

In [ ]:
boe_portfolio[boe_portfolio['ISIN']==unsold[0]]

In [ ]:
assert boe_portfolio['Total allocation (nominal £mn)'].sum() == check_sales

In [ ]:
boe_portfolio['current_position'] = boe_portfolio['Total_Allocation_Nominal'] - boe_portfolio['Total allocation (nominal £mn)']
boe_portfolio['position_cost'] = boe_portfolio['Total_Allocation_Proceeds'] - boe_portfolio['Total allocation (proceeds £mn)']

In [ ]:
boe_portfolio['Coupon']=boe_portfolio['ISIN'].map(
    lambda x: isin_bond_details[
        isin_bond_details['ISIN Code']==x
        ]['%'].values[0]
    )

In [ ]:
boe_portfolio.groupby('Coupon')['current_position'].sum().plot(
    kind='bar', figsize=(12, 6), title='BoE Portfolio by Coupon'
)

In [ ]:
boe_portfolio.groupby('Coupon')['position_cost'].sum().cumsum()

In [ ]:
from bgs.load_bgs_prices import load_prices
price_df=load_prices("downloads/BGSPrices.csv")

In [ ]:
price_df.index = pd.to_datetime(price_df.index, format="%d %b %Y").to_period('M').to_timestamp('M')

In [ ]:
price_df.loc['Apr 2025',32100]

In [ ]:
boe_portfolio['mkt_price'] = boe_portfolio['bgs_id'].map(
    lambda x: price_df.loc['Apr 2025', int(x)].values[0])

In [ ]:
boe_portfolio.head()

In [ ]:
boe_portfolio['p&l'] = boe_portfolio['mkt_price'] * boe_portfolio['current_position']/100 - boe_portfolio['position_cost']  

In [ ]:
boe_portfolio['p&l'].sum()

In [ ]:
boe_portfolio.groupby('Coupon')['p&l'].sum().plot(
    kind='bar', figsize=(12, 6), title='BoE Portfolio Loses by Coupon'
)

In [ ]:
boe_portfolio.groupby('Coupon')['p&l'].sum().cumsum()

In [ ]:
from bgs.load_bgs_amounts import load_bgs_amounts
tables = load_bgs_amounts("downloads/BGSAmounts.csv")

In [ ]:
conv_amt = tables['Conventionals']

In [ ]:
conv_amt.loc['30 Apr 2025','32100']

In [ ]:
boe_portfolio['tot_outstanding'] = boe_portfolio['bgs_id'].map(
    lambda x: conv_amt.loc['30 Apr 2025', x])

In [ ]:
boe_portfolio.head()

In [ ]:
boe_portfolio['pct_ownership']=boe_portfolio['current_position']/boe_portfolio['tot_outstanding'].astype(float).fillna(0)

In [ ]:
boe_portfolio['pct_ownership'].plot(kind='hist')

In [ ]:
boe_portfolio.plot(x='pct_ownership',y='p&l', kind='scatter')

In [ ]:
boe_portfolio[boe_portfolio['pct_ownership']>0.5][['ISIN','p&l', 'pct_ownership']].sort_values(by='p&l', ascending=True)

In [ ]:
conv_details.loc[conv_details['ISIN Code']=='GB00BFWFPP71'].T

In [ ]:
boe_end_purchase_port =  gilt_buys_total.loc[gilt_buys_total['maturity_date'] > gilt_buys_total['Last_Purchase_Month'].max()]

In [ ]:
boe_end_purchase_port['year'] = boe_end_purchase_port['maturity_date'].dt.year
boe_end_purchase_port.groupby('year')['Total_Allocation_Nominal'].sum().cumsum().plot(kind='bar', x='year', y='Total_Allocation_Nominal')